In [1]:
import sys

sys.path.append('../..')

In [2]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
mpor = 15*step

#Discount

In [3]:
from finance.discountfactor import ConstantRateDiscountFactor 

maturity = 5.
r = 0.02

discount = ConstantRateDiscountFactor(r)

# Exponential distrib

In [4]:
from scipy.stats import expon
 
mean = maturity / 2.
exp_distrib = expon(loc=0, scale=mean)

print "Mean = %s"%mean

Mean = 2.5


# Underlying

In [5]:
import numpy as np
from maths.montecarlo.processes.brownianmotions import BrownianMotion

time_grid = BrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100]
drift = 6
vol = 15

b = BrownianMotion(time_grid, x_0, drift, vol)

# Derivative

In [6]:
from finance.products.european.assetswap import (
    SwapContract,
)

delta = 0.5

swap_dates = SwapContract.generate_payment_dates(0, maturity, delta)
swap = SwapContract(b, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 = ",price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 104.560, paying at {0.00, 0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00}

Price swap at t=0 =  -5.68434188608e-14


# Exposure

In [7]:
from risk.exposures import EuropeanQuantileBrownianExposure

quantile_im = 0.80

exposure = EuropeanQuantileBrownianExposure(swap, discount, drift, vol)

# Load intensities of obligors

In [8]:
import dill

raw_intensities = None
with open('../intensities/intensities.pkl', 'rb') as f:
    raw_intensities = dill.load(f)
    
raw_intensities.keys()

['groups', 'idio']

In [9]:
raw_idios = np.array(raw_intensities['idio'])
raw_groups = np.array(raw_intensities['groups'])

In [10]:
subsets = []
hazard_rates = []
pillars = []

for d_idio in raw_idios:
    subsets.append(d_idio['id'])
    hazard_rates.append(d_idio['hazard_rates'])
    pillars.append(d_idio['pillars'])
    
for d_groups in raw_groups:    
    subsets.append(d_groups['ids'])
    hazard_rates.append(d_groups['hazard_rates'])
    pillars.append(d_groups['pillars'])

#Copula

In [11]:
from maths.copula.marshallolkin import StepWiseIntensitiesMarshallOlkinCopula

copula = StepWiseIntensitiesMarshallOlkinCopula(subsets, hazard_rates, pillars)

# IDs parameters

In [12]:
bank_index = 113
counterparty_index = 26

# Portfolio construction

In [13]:
p_fixed = 100.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

In [14]:
from finance.portfolio import EquilibratedPortfolio

total_nb_of_banks = len(raw_groups[-2]['ids'])
pos = EquilibratedPortfolio.generate_1_vs_all_positions(bank_index, raw_groups[-2]['ids'], len(raw_idios))

positions = np.array(pos*notional).reshape(pos.size, 1)

port = EquilibratedPortfolio(positions, [swap], [exposure])

# VM and IM accounts

In [15]:
from ccp.accounts import Accounts
from ccp.states import MembersState

derivatives_nb = 1

states = MembersState(raw_idios.size)
vm_accounts = Accounts(states, derivatives_nb)
im_accounts = Accounts(states, derivatives_nb)

In [16]:
# Be carefull, here, as the default models will be 
# used for regulatory computations, we select only 
# the first 125 idiosyncratic models

default_proba_models = copula.models[0 : raw_idios.size]

# Regulatory capital

In [17]:
from risk.basel import RegulatoryCapital

recoveries = [0.4 for i in range(0, raw_idios.size)]

regul_cap = RegulatoryCapital(vm_accounts, im_accounts, port, default_proba_models, recoveries, bank_index=bank_index)

# Funding and capital parameters

In [18]:
lambda_ = 0.
recov_funding = 1.
k_ = 0.1

#MC loop

In [19]:
def time_offseter(time, time_grid_ref, left=False):
    offset = -1 if left else 0
    idx = np.searchsorted(time_grid_ref, time, side='right') + offset
    
    if idx == len(time_grid):        
        return np.inf
    
    return time_grid_ref[idx]

In [20]:
results = {"cva": {"sum": 0., "sum2": 0.},
           "dva": {"sum": 0., "sum2": 0.},
           "fva": {"sum": 0., "sum2": 0.},
           "kva_ccr": {"sum": 0., "sum2": 0.},
           "kva_cva": {"sum": 0., "sum2": 0.}}

In [21]:
N = 1000

In [22]:
import time

tic = time.time()

zetas = exp_distrib.rvs(size=N)

b_subsets_indexes = copula.get_indexes_including(bank_index)
c_subsets_indexes = copula.get_indexes_including(counterparty_index)
b_and_c_subsets_indexes = list(set(b_subsets_indexes).union(c_subsets_indexes))
b_and_c_subsets_indexes.sort()

default_times_mat = copula.generate_default_times(subsets_indexes=b_and_c_subsets_indexes, number=N)

b_default_times_indexes = [ii for ii, b_index in enumerate(b_and_c_subsets_indexes) if b_index in b_subsets_indexes]
c_default_times_indexes = [ii for ii, c_index in enumerate(b_and_c_subsets_indexes) if c_index in c_subsets_indexes]

for i in range(N):
    # Resurrecting the states
    # reset to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    
    zeta = time_offseter(zetas[i], time_grid)
    default_times = default_times_mat[i]
        
    tau = default_times.min()
    bar_tau = min(tau, maturity)
    
    if bar_tau <= zeta:
        continue
    
    inv_pdf = 1./exp_distrib.pdf(zeta)
    discount_ratio = discount(zeta+mpor) / discount(zeta)
    mult_factor = inv_pdf*discount_ratio
        
    zeta_delta = time_offseter(zeta+mpor, time_grid, True)    
    
    keeping_pillars = swap.pillars[swap.pillars <= zeta_delta]
    step_time_grid = np.append(keeping_pillars, [zeta, zeta_delta])
    step_time_grid.sort()
    
    # We generate the brownian motion
    # only on these new times 
    #
    # Setting time calls simulate at 
    # the end of the method
    b.time = step_time_grid
    
    # IM computations
    b_im = port.compute_exposure(zeta, risk_period=mpor, conf_level=quantile_im, \
                                 from_=bank_index, towards_=counterparty_index)
    c_im = port.compute_exposure(zeta, risk_period=mpor, conf_level=quantile_im, \
                                 from_=counterparty_index, towards_=bank_index)
    
    im_accounts.put_amounts(bank_index, b_im)
    im_accounts.put_amounts(counterparty_index, c_im)
    
    d_pl = [s.price(zeta) for s in [swap]]
    b_vm = port.compute_value(d_pl, from_=bank_index, towards_=counterparty_index)    
    vm_accounts.put_amounts(bank_index, b_vm)
    vm_accounts.put_amounts(bank_index, -b_vm)
    p_zeta = b_vm.sum()
    
    d_pl = [s.price(zeta_delta) for s in [swap]]
    p_zeta_delta = port.compute_value(d_pl, from_=bank_index, towards_=counterparty_index).sum()
    q_zeta_delta = p_zeta_delta
        
    cva_ = 0.
    dva_ = 0.
    fva_ = 0.
    kccr_ = 0.
    kcva_ = 0.
    for j, (subset_index, eta) in enumerate(zip(b_and_c_subsets_indexes, default_times)):
        gamma_zeta = copula.gamma(subset_index, zeta)
        
        ####################
        ##### CVA part #####
        ####################
        c_default_indic = j in c_default_times_indexes
        if not c_default_indic:            
            c_default_times = default_times[c_default_times_indexes]
            c_min_default_time = c_default_times.min()
            c_default_indic = c_min_default_time<=zeta_delta
        
        if c_default_indic:            
            b_collat = p_zeta + im_accounts.get_amount(bank_index).sum()
            loss = np.maximum(q_zeta_delta-b_collat, 0.)
            cva_ += mult_factor*gamma_zeta*(1.-recoveries[counterparty_index]*loss)            
            
        ####################
        ##### DVA part #####
        ####################
        b_default_indic = j in b_default_times_indexes
        if not b_default_indic:            
            b_default_times = default_times[b_default_times_indexes]
            b_min_default_time = b_default_times.min()
            b_default_indic = b_min_default_time<=zeta_delta
            
        if b_default_indic:
            c_collat = p_zeta - im_accounts.get_amount(counterparty_index).sum()
            loss = np.maximum(q_zeta_delta-c_collat, 0.)
            dva_ += mult_factor*gamma_zeta*(1.-recoveries[bank_index]*loss)
                        
        ####################
        ##### FVA part #####
        ####################
        b_funded_collat = p_zeta + \
                          im_accounts.get_amount(bank_index).sum() - \
                          im_accounts.get_amount(counterparty_index).sum()
                
        b_gamma_zeta = copula.tot_gamma(bank_index, zeta)
        
        bar_lambda_zeta = .5*(1.-recoveries[bank_index])*b_gamma_zeta
        
        tilde_lambda_zeta = bar_lambda_zeta - (1.-recov_funding)*b_gamma_zeta
        c_zeta = -.0020 + bar_lambda_zeta
        
        lambda_zeta = lambda_
        
        fva_ += mult_factor * b_gamma_zeta * (c_zeta*b_funded_collat + \
                                             tilde_lambda_zeta*np.maximum(p_zeta-b_funded_collat, 0.) + \
                                             lambda_zeta*np.minimum(p_zeta-b_funded_collat, 0.))
                
        ######################
        ##### K_CCR part #####
        ######################        
        kccr_ += mult_factor*k_*regul_cap.compute_kccr(counterparty_index, zeta)
                
        ######################
        ##### K_CCR part #####
        ######################        
        kcva_ += mult_factor*k_*regul_cap.compute_kcva(counterparty_index, zeta)
        
    results['cva']['sum'] += cva_
    results['cva']['sum2'] += cva_**2
    
    results['dva']['sum'] += dva_
    results['dva']['sum2'] += dva_**2
    
    results['fva']['sum'] += fva_
    results['fva']['sum2'] += fva_**2
        
    results['kva_ccr']['sum'] += kccr_
    results['kva_ccr']['sum2'] += kccr_**2
    
    results['kva_cva']['sum'] += kcva_
    results['kva_cva']['sum2'] += kcva_**2
    
toc = time.time()

#Results

In [23]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%bank_index
for idx in b_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], copula.pillars[idx], copula.intensities[idx])
    
print "Counterparty index %i that belongs to the following MO copula subsets:"%counterparty_index
for idx in c_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], copula.pillars[idx], copula.intensities[idx])
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
for lab, pos in zip(["bank", "counterparty"], [bank_index, counterparty_index]):
    print "- %s: %s"%(lab, port.positions[pos, :])
    
print "\nConfidence level used for IM: %.2f"%quantile_im

print "\nResults of the computation:\n"
keys = results.keys()
keys.sort()

res = {}

for k in keys:
    v = results[k]
    mean = v['sum']/N
    mod_var = (v['sum2']/N - mean**2) / (N-1)
    half_inter = z_level*np.sqrt(mod_var)
    
    res[k] = [mean-half_inter, mean+half_inter]
    
    print "The %s for the bank is: [%f, %f]"%(k.upper(), mean-half_inter, mean+half_inter)

Results for 1000 iterations (15.0169999599 secs.)

Used discount factor: Constant discount factor process with rate r = 0.02

Bank index 113 that belongs to the following MO copula subsets:
- [113] with pillars [3 5] and intensity [ 0.00374407  0.00339649]

- [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124] with pillars [3 5] and intensity [  2.01650000e-04   9.33720000e-05]

- [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104

In [24]:
results

{'cva': {'sum': 503.16136448480728, 'sum2': 447.78408480675989},
 'dva': {'sum': 24.31952474121584, 'sum2': 1.9706846418020096},
 'fva': {'sum': 0.0098262319006385858, 'sum2': 3.9086763706276565e-06},
 'kva_ccr': {'sum': 384.50573116224751, 'sum2': 257.96236523765452},
 'kva_cva': {'sum': 463.35998780276191, 'sum2': 395.34676205932186}}

In [25]:
import dill

with open('results_%svs%s.pkl'%(bank_index, counterparty_index), 'wb') as f:
    to_save = {"bank_index": bank_index, 
               "counterparty_index": counterparty_index,
               "results": res}
    
    dill.dump(to_save, f)